# Carbon System Data Analysis

---
### Purpose
The purpose of this notebook is to analyze the performance of the Sunburst Sensors, LLC. SAMI-pH (PHSEN) pH and SAMI-pCO$_{2}$ seawater measurements, and the Pro-Oceanus pCO2 sensor measurements at the Pioneer Coastal Array and Global Irminger Array. This is done on a deployment-by-deployment, site-by-site comparison with the pH measurements from discete water samples collected by Niskin Bottle casts during deployment and recovery of the instrumentation during mooring maintainence. 

---
### Datasets
There are three main sources of data sources:
* **Deployments**: These are the deployment master sheets from OOI Asset Management. They contain the deployment numbers, deployment start times and cruise, and recovery times and cruise, for all of the instrumentation deployed. 
* **PHSEN**: This is the Sunburst Sensors, LLC. SAMI-pH sensor. It is calibrated for pH values from 70-9.0 pH units for salinities from 25-40 psu. Manufacturers stated accuracy of 0.003 pH units, precision < 0.001 pH units, and long-term drift of < 0.001 pH units / 6 months. The data is downloaded from the Ocean Observatories data portal (OOINet) as netCDF files.
* **PCO2W**: This is the Sunburst Sensors, LLC. SAMI-CO<sub>2</sub> sensor. It is calibrated for pCO<sub>2</sub> concentrations from 200 - 1000 ppm. Manufacturers stated accuracy of 3 ppm, precision < 1 ppm, and long-term drift of < 1 ppm / 6 months. The data is downloaded from the Ocean Observatories data portal (OOINet) as netCDF files.
* **CTDBP**: This is the collocated SeaBird CTD with the PHSEN. The data is downloaded from the Ocean Observatories data portal (OOINet) as netCDF files. These data are needed since the PCO2W datasets do not contain either temperature (T), salinity (S), pressure (P), or density ($\rho$) data needed to compare with the discrete sampling.
* **Discrete Water Samples**: These are discrete water samples collected via Niskin Bottle casts during deployment and recovery of the moored instrumentation. The data is downloaded from OOI Alfresco website as excel files. Parameters sampled include oxygen, salinity, nutrient concentrations (phosphate, nitrate, nitrite, ammonium, silicate), chlorophyll concentrations, and the carbon system. The carbon system parameters sampled are Total Alkalinity (TA), Dissolved Inorganic Carbon (DIC), and pH. 
---
### Method
#### PCO2W Processing
Verifying the in-situ pCO<sub>2</sub> measured by the PCO2W against the pCO<sub>2</sub> calculated from the discrete water samples TA and DIC requires several preprocessing steps of the PCO2W datasets. First, the netCDF datasets are opened using ```xarray``` into an xarray ```dataset``` object and the primary dimension set to 'time'. Next, T, S, P, and $\rho$ are interpolated to the PCO2W time base using xarray ```ds.interp_like``` from the dataset from the collocated CTDBP. Next, the pCO2 is corrected for hydrostatic pressure using a correction of 15% per 1000 dbar pressure (Enns 1965, Reed et al. 2018). Then the first and last four days of PCO2W data are selected. The standard deviation of the selected pCO2 is calculated using the first-order differencing with a time-lag of one, in order to arrive at a quasi-stationary time series.

#### CTDBP Processing
The associated CTD datasets to the PCO2W are opened using ```xarray``` into an xarray ```dataset``` object and the primary dimension set to 'time'. The CTD dataset T, S, P, and $\rho$ are interpolated to the PCO2W time base and merged into the PCO2W dataset using ```ds.interp_like```. 

#### Discrete Water Samples Processing
The relevant deployment and recovery cruise data for comparison with the PCO2W dataset(s) are opened and loaded into a pandas ```DataFrame``` object. Next, the pCO<sub>2</sub> concentrations are calculated using the ```CO2SYS``` package from the associated TA and DIC concentrations. The bottle samples are then filtered by cruise, time, and depth to identify the samples associated with the deployment and recovery of the PCO2W dataset being analyzed.

In [1]:
import os, sys, gc
import json
import yaml
import numpy as np
import pandas as pd
import xarray as xr
import warnings
warnings.filterwarnings("ignore")

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# Import the OOI M2M tool
sys.path.append("/home/andrew/Documents/OOI-CGSN/ooinet/ooinet/")
from m2m import M2M

#### Set OOINet API access
In order access and download data from OOINet, need to have an OOINet api username and access token. Those can be found on your profile after logging in to OOINet. Your username and access token should NOT be stored in this notebook/python script (for security). It should be stored in a yaml file, kept in the same directory, named user_info.yaml.

In [4]:
# Import user info for connecting to OOINet via M2M
userinfo = yaml.load(open("../../../../QAQC_Sandbox/user_info.yaml"), Loader=yaml.FullLoader)
username = userinfo["apiname"]
token = userinfo["apikey"]

#### Connect to OOINet

In [5]:
OOINet = M2M(username, token)

---
## Discrete Bottle Data

#### Irminger Sea
First, load the bottle data from the Irminger Sea Array into a pandas dataframe. Once the data is loaded, then we'll do some datatype conversions, such as converting the Dates and Times into pandas datetime objects, as well as replacing the fill value of ```-9999999``` with NaNs. Finally, we'll drop the data from the table that doesn't have any DIC, Alkalinity, or pH data to get the relevant carbon system data we'll need for comparisons.

We'll also want to check the associated bottle and discrete sample flags. In particular, we'll want to check for ```Niskin Flag``` with a flag of ```*0000000000001000``` which indicates a leaking bottle, and the ```Discrete TA/DIC/pH Flag``` for entries of either ```*0000000000000010```, which indicates the sample analysis is not yet complete, or ```*0000000000001000``` / ```*0000000000010000``` which indicate suspicious or bad data. We'll translate these flags into the standard WOCE schema of:
* 1 = good data
* 2 = no evaluation
* 3 = questionable data
* 4 = bad data
* 9 = missing/no data

In [6]:
bottleData = pd.read_csv("../data/water_sampling/Irminger_Bottle_Data.csv")
bottleData

,Cruise,Station,Target Asset,Start Latitude [degrees],Start Longitude [degrees],Start Time [UTC],Cast,Cast Flag,Bottom Depth at Start Position [m],CTD File,...,Calculated Bicarb [umol/kg],Calculated CO3 [umol/kg],Calculated Omega-C,Calculated Omega-A,CTD Absolute Salinity [g/kg],CTD Conservative Temperature,CTD Density [kg/m^3],CTD Sigma [kg/m^3],Calculated pCO2 [lab conditions],Calculated pH [lab conditions]
0,AR35-05,8,Gliders,59.844167,-39.094567,2019-08-10 20:33:00+00:00,8,*0000000000000001,2824.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AR35-05,8,Gliders,59.844167,-39.094567,2019-08-10 20:33:00+00:00,8,*0000000000000001,2824.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AR35-05,8,Gliders,59.844167,-39.094567,2019-08-10 20:33:00+00:00,8,*0000000000000001,2824.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AR35-05,9,"GI01SUMO, GI02HYPM",59.965767,-39.565850,2019-08-11 16:43:00+00:00,9,*0000000000000001,2654.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AR35-05,9,"GI01SUMO, GI02HYPM",59.965767,-39.565850,2019-08-11 16:43:00+00:00,9,*0000000000000001,2654.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
877,AR30-03,23,CL2,62.058500,-32.231833,2018-06-22 20:07:00+00:00,23,*0000000000000100,2400.0,D:\Data\ar30-03023.hex,...,NaN,NaN,NaN,NaN,35.050344,5.174898,1028.042319,1028.042319,NaN,NaN
878,AR30-03,23,CL2,62.058500,-32.231833,2018-06-22 20:07:00+00:00,23,*0000000000000100,2400.0,D:\Data\ar30-03023.hex,...,NaN,NaN,NaN,NaN,35.052420,6.910300,1027.540094,1027.540094,NaN,NaN
879,AR30-03,23,CL2,62.058500,-32.231833,2018-06-22 20:07:00+00:00,23,*0000000000000100,2400.0,D:\Data\ar30-03023.hex,...,NaN,NaN,NaN,NaN,35.054759,7.243608,1027.416413,1027.416413,NaN,NaN
880,AR30-03,23,CL2,62.058500,-32.231833,2018-06-22 20:07:00+00:00,23,*0000000000000100,2400.0,D:\Data\ar30-03023.hex,...,NaN,NaN,NaN,NaN,35.048232,7.269356,1027.370393,1027.370393,NaN,NaN


With the flags interpreted to simpler scheme, can now filter the data to keep only the "good" and "suspicious" data. We'll replace the bad or missing data with NaNs so we can drop them in the next step

In [7]:
# ====================================================
# Irminger Sea data
mask = (bottleData["Discrete Alkalinity Flag"] != 1) & (bottleData["Discrete Alkalinity Flag"] != 3)
to_replace = bottleData["Discrete Alkalinity [umol/kg]"][mask].dropna().unique()
bottleData["Discrete Alkalinity [umol/kg]"].replace(to_replace=to_replace, value=np.nan, inplace=True)

mask = (bottleData["Discrete DIC Flag"] != 1) & (bottleData["Discrete DIC Flag"] != 3)
to_replace = bottleData["Discrete DIC [umol/kg]"][mask].dropna().unique()
bottleData["Discrete DIC [umol/kg]"].replace(to_replace=to_replace, value=np.nan, inplace=True)

mask = (bottleData["Discrete pH Flag"] != 1) & (bottleData["Discrete pH Flag"] != 3)
to_replace = bottleData["Discrete pH [Total scale]"][mask].dropna().unique()
bottleData["Discrete pH [Total scale]"].replace(to_replace=to_replace, value=np.nan, inplace=True)

Drop rows with NaNs in the carbon system data, since those don't have any relevant data

In [8]:
carbon_columns = ["Discrete Alkalinity [umol/kg]", "Discrete DIC [umol/kg]", "Discrete pH [Total scale]"]
carbonData = bottleData.dropna(subset=carbon_columns, how="all")

## PCO2A

First, we examine the **```PCO2A```** dataset from the Irminger Surface Mooring Buoy. The **```PCO2A```** samples both the air and the surface water pCO2 concentrations. 

In [ ]:
sys.path.append("../../")
from OS2022.OS2022 import plotting

In [ ]:
# Get the dataset
pco2a = xr.open_dataset("../data/CP04OSSM_SBD12_04_PCO2AA000.nc")
pco2a

In [ ]:
# Plot the data as a function of deployments
fig, ax = plotting.plot_data_variable(pco2a, "partial_pressure_co2_ssw")
ax.set_ylim((200, 500))

### Annotations
Next, add in the annotations to the dataset and filter out the data identified as bad

In [ ]:
annotations = OOINet.get_annotations("CP04OSSM-SBD12-04-PCO2AA000")
annotations

From the plot of the PCO2A, we can see that the instrument failed before recovery each time that it was deployed. Next, we want to add the annotation information to the OOI data

#### Add Annotation Information to dataset

In [ ]:
pco2a = OOINet.add_annotation_qc_flag(pco2a, annotations)
pco2a

In [ ]:
np.unique(pco2a.rollup_annotations_qc_results)

Plot the annotation data

In [ ]:
#fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(12, 8))
#ax.plot(pco2a.time, pco2a.partial_pressure_co2_ssw, marker=".", linestyle="", color="tab:blue")
# Plot the data as a function of deployments
fig, ax = plotting.plot_data_variable(pco2a, "partial_pressure_co2_ssw")
ax.plot(pco2a.time.where(pco2a.rollup_annotations_qc_results == 9, drop=True),
        pco2a.partial_pressure_co2_ssw.where(pco2a.rollup_annotations_qc_results == 9, drop=True),
        marker="o", linestyle="", color="tab:red")
ax.plot(pco2a.time.where(pco2a.rollup_annotations_qc_results == 3, drop=True),
        pco2a.partial_pressure_co2_ssw.where(pco2a.rollup_annotations_qc_results == 3, drop=True),
        marker="o", linestyle="", color="tab:orange")
ax.set_xlabel("time")
ax.set_ylabel(pco2a.partial_pressure_co2_ssw.attrs["long_name"])
ax.set_ylim((200, 500))
ax.grid()

#### Filter Annotations
Finally, remove the manually-identified bad data from the dataset

In [ ]:
pco2a = pco2a.where(pco2a.rollup_annotations_qc_results != 9, drop=True)
pco2a

In [ ]:
fig, ax = plotting.plot_data_variable(pco2a, "partial_pressure_co2_ssw")
ax.set_ylim((200, 500))

In [ ]:
pco2a.lat, pco2a.lon

### Identify discrete samples
Next, identify the discrete water samples associated with the Irminger Surface Buoy PCO2A. The surface samples should be less than 

In [ ]:
!pip install haversine

In [ ]:
import haversine as hs

In [ ]:
# For each
loc1 = (59.9337, -39.47378)
loc2 = (59.94358, -39.57372)

In [ ]:
def findNearest(bottleData, buoyLoc, maxDist):
    """Find the bottle sample values within a maximum distance from the buoy
    
    Parameters
    ----------
    bottleData: (pd.DataFrame -> strings or floats)
        A tuple of (latitude, longitude) values in decimal degrees of the bottle sample location
    buoyLoc: (tuple -> floats)
        A tuple of (latitude, longitude) values in decimal degrees of the buoy location
    maxDist: (float)
        Maximum distance in km away for a sample location from the buoy location
    
    Returns
    -------
    mask: (boolean)
        Returns True or False boolean if sampleLoc < maxDist from buoyLoc
    """
    # Get the startLat/startLon as floats
    startLat = pioneer["Start Latitude [degrees]"].apply(lambda x: float(x))
    startLon = bottleData["Start Longitude [degrees]"].apply(lambda x: float(x))
    
    # Calculate the distance
    distance = []
    for lat, lon in zip(startLat, startLon):
        sampleLoc = (lat, lon)
        distance.append(hs.haversine(sampleLoc, buoyLoc))
    
    # Filter the results
    return [d <= maxDist for d in dist]

In [ ]:
def findSamples(bottleData, buoyLoc, maxDist, depthTol):
    
    """Find the bottle sample values within a maximum distance from the buoy
    
    Parameters
    ----------
    bottleData: (pd.DataFrame -> strings or floats)
        A tuple of (latitude, longitude) values in decimal degrees of the bottle sample location
    buoyLoc: (tuple -> floats)
        A tuple of (latitude, longitude) values in decimal degrees of the buoy location
    maxDist: (float)
        Maximum distance in km away for a sample location from the buoy location
    
    Returns
    -------
    mask: (boolean)
        Returns True or False boolean if sampleLoc < maxDist from buoyLoc

In [ ]:
pco2a

In [ ]:
distMask = findNearest(pioneer, (pco2a.lat, pco2a.lon), 0.5)
pioneer[distMask]

In [ ]:
[x for x in hs.Unit if x.value == distUnit]

In [ ]:
startLat = pioneer["Start Latitude [degrees]"].apply(lambda x: float(x))
startLon = pioneer["Start Longitude [degrees]"].apply(lambda x: float(x))
dist = []
buoyLoc = (pco2a.lat, pco2a.lon)
for lat, lon in zip(startLat, startLon):
    sampleLoc = (lat, lon)
    dist.append(hs.haversine(sampleLoc, buoyLoc))


In [ ]:
[d <= 0.5 for d in dist]

In [ ]:
pco2a.lon

In [ ]:
# Identify samples associated with the instrument using the "target asset"
mask = pioneer["Target Asset"].apply(lambda x: True if "ossm" in x.lower() else False)
ossm_bottle_data = pioneer[mask]

In [ ]:
# Identify samples from near the surface, which we'll take as <= 15 dbar of pressure
mask = ossm_bottle_data["CTD Pressure [db]"] <= 15
ossm_pco2a_bottle_data = ossm_bottle_data[mask]
ossm_pco2a_bottle_data

In [ ]:
ossm_pco2a_bottle_data["Calculated pCO2 [uatm]"]

In [ ]:
# Plot the bottle data on top of the pco2a data
fig, ax = plotting.plot_data_variable(pco2a, "partial_pressure_co2_ssw")

ax.plot(ossm_pco2a_bottle_data["CTD Bottle Closure Time [UTC]"], ossm_pco2a_bottle_data["Calculated pCO2 [uatm]"],
        marker="o", color="tab:red", linestyle="", markersize=12)
ax.set_ylim((200, 500))

From the above plot of the discrete bottle samples against the PCO2A, we can select a couple of deployments for a closer analysis of the pCO2 calculated from the DIC/TA relationship and the measured seawater pCO2. We'll choose Deployments 3 and 7 to look at more closely.

In [ ]:
pco2a_deployment_3 = pco2a.where(pco2a.deployment == 3, drop=True)

Plot deployment 3 more closely

In [ ]:
fig, ax = plotting.plot_data_variable(pco2a_deployment_3, "partial_pressure_co2_ssw", add_deployments=False)
xlim = ax.get_xlim()

# Calculate the noise level
std = pco2a_deployment_3.partial_pressure_co2_ssw.diff(dim="time").std().values

ax.fill_between(pco2a_deployment_3.time,
                pco2a_deployment_3.partial_pressure_co2_ssw-2*std,
                pco2a_deployment_3.partial_pressure_co2_ssw+2*std,
                color="tab:blue")
ax.plot(ossm_pco2a_bottle_data["Start Time [UTC]"], ossm_pco2a_bottle_data["Calculated pCO2 [uatm]"], 
        color="tab:red", marker="*", linestyle="", markersize=12)
ax.set_xlim(xlim)

In [ ]:
# Write a function which takes in a time series and bottle values and plot the data comparisons

In [ ]:
deployments = np.unique(pco2a.deployment)
deployments

In [ ]:
results = pd.DataFrame()
for deployment in deployments:
    # First, select the relevant deployment data
    deployment_data = pco2a.where(pco2a.deployment == deployment, drop=True)
    
    # Get the refdes
    refdes = "-".join(pco2a.attrs["id"].split("-")[0:4])
    
    # Next, get the first and last timestamps of the pco2a data
    deployDT, recoverDT = deployment_data.time.min().values, deployment_data.time.max().values
    
    # Calculate the mean and std deviation for the first day of data from the instrument
    deploy_avg = deployment_data.partial_pressure_co2_ssw.where(deployment_data.time <= pd.to_datetime(deployDT + pd.Timedelta("1D"))).mean()
    deploy_std = deployment_data.partial_pressure_co2_ssw.where(deployment_data.time <= pd.to_datetime(deployDT + pd.Timedelta("1D"))).std()

    # Calculate the mean and std devation for the last day of the data from the instrument
    recover_avg = deployment_data.partial_pressure_co2_ssw.where(deployment_data.time >= pd.to_datetime(recoverDT - pd.Timedelta("1D"))).mean()
    recover_std = deployment_data.partial_pressure_co2_ssw.where(deployment_data.time >= pd.to_datetime(recoverDT - pd.Timedelta("1D"))).std()

    # Now, filter for the nearest bottle data based on start and stop times
    deploy_bottles = ossm_pco2a_bottle_data[
        (ossm_pco2a_bottle_data["Start Time [UTC]"] >= pd.to_datetime(deployDT - pd.Timedelta("1D"), utc=True)) &
        (ossm_pco2a_bottle_data["Start Time [UTC]"] <= pd.to_datetime(deployDT + pd.Timedelta("1D"), utc=True))]
    recover_bottles = ossm_pco2a_bottle_data[
        (ossm_pco2a_bottle_data["Start Time [UTC]"] >= pd.to_datetime(recoverDT - pd.Timedelta("1D"), utc=True)) &
        (ossm_pco2a_bottle_data["Start Time [UTC]"] <= pd.to_datetime(recoverDT + pd.Timedelta("1D"), utc=True))]
    
    # Whittle down the Deployment Data to the most relevant values
    columns=["CTD Pressure [db]", "CTD Absolute Salinity [g/kg]", "CTD Conservative Temperature", "Calculated pCO2 [uatm]"]
    
    
    # Okay, now calculate the comparison between the bottle and
    results = results.append(deploy_bottles)
    

In [ ]:
deployDT, recoverDT = pco2a_deployment_3.time.min().values, pco2a_deployment_3.time.max().values
deployDT, recoverDT

In [ ]:
refdes = "-".join(pco2a.attrs["id"].split("-")[0:4])
refdes

In [ ]:
pd.to_datetime(deployDT - pd.Timedelta("1D"), utc=True)

In [ ]:
deploy_bottles = ossm_pco2a_bottle_data[
    (ossm_pco2a_bottle_data["Start Time [UTC]"] >= pd.to_datetime(deployDT - pd.Timedelta("1D"), utc=True)) &
    (ossm_pco2a_bottle_data["Start Time [UTC]"] <= pd.to_datetime(deployDT + pd.Timedelta("1D"), utc=True))]

recover_bottles = ossm_pco2a_bottle_data[
    (ossm_pco2a_bottle_data["Start Time [UTC]"] >= pd.to_datetime(recoverDT - pd.Timedelta("1D"), utc=True)) &
    (ossm_pco2a_bottle_data["Start Time [UTC]"] <= pd.to_datetime(recoverDT + pd.Timedelta("1D"), utc=True))]

In [ ]:
deploy_bottles[["CTD Pressure [db]", "CTD Absolute Salinity [g/kg]", "CTD Conservative Temperature", "Calculated pCO2 [uatm]"]]

In [ ]:
# Calculate the comparison values
deploy_avg = pco2a_deployment_3.partial_pressure_co2_ssw.where(pco2a_deployment_3.time <= pd.to_datetime(deployDT + pd.Timedelta("1D"))).mean()
deploy_std = pco2a_deployment_3.partial_pressure_co2_ssw.where(pco2a_deployment_3.time <= pd.to_datetime(deployDT + pd.Timedelta("1D"))).std()
deploy_avg, deploy_std

In [ ]:
deploy_bottles["Calculated pCO2 [uatm]"]

In [ ]:
# Calculate the comparison values
recover_avg = pco2a_deployment_3.partial_pressure_co2_ssw.where(pco2a_deployment_3.time >= pd.to_datetime(recoverDT - pd.Timedelta("1D"))).mean()
recover_std = pco2a_deployment_3.partial_pressure_co2_ssw.where(pco2a_deployment_3.time >= pd.to_datetime(recoverDT - pd.Timedelta("1D"))).std()
recover_avg, recover_std

In [ ]:
recover_bottles["Calculated pCO2 [uatm]"]

In [ ]:
from statsmodels.tsa.stattools import adfuller

def adfuller_test(sales):
    result=adfuller(sales)
    labels = ['ADF Test Statistic','p-value','#Lags Used','Number of Observations Used']
    for value,label in zip(result,labels):
        print(label+' : '+str(value) )
    if result[1] <= 0.05:
        print("P value is less than 0.05 that means we can reject the null hypothesis(Ho). Therefore we can conclude that data has no unit root and is stationary")
    else:
        print("Weak evidence against null hypothesis that means time series has a unit root which indicates that it is non-stationary ")

In [ ]:
adfuller_test(pco2a_deployment_3["partial_pressure_co2_ssw"])

In [ ]:
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [ ]:
fig = plt.figure(figsize=(12, 8))
ax1 = fig.add_subplot(2,1,1)
fig = plot_acf(pco2a_deployment_3.partial_pressure_co2_ssw.diff(dim="time"), lags=40, ax=ax1)
ax2 = fig.add_subplot(2,1,2)
fig = plot_pacf(pco2a_deployment_3.partial_pressure_co2_ssw.diff(dim="time"), lags=40, ax=ax2)

We'll calculate the noise term using a first-difference estimate to get the standard deviation of the data

---
## Instrument Datasets

### Irminger Array
* GI01SUMO: Apex Surface Mooring
    * SBD12: Surface Buoy
        * PCO2AA: pCO2 Air-Sea (refdes = GI01SUMO-SBD12-04-PCO2AA000)
        * METBKA: Bulk Meteorology Instrument Package (refdes = GI01SUMO-SBD12-06-METBKA000)
    * RID16: Near-Surface Instrument Frame
        * PCO2WB: pCO2 Water (refdes = GI01SUMO-RID16-05-PCO2WB000)
        * CTDBPF: CTD (refdes = GI01SUMO-RID16-03-CTDBPF000)
    * RII11: Mooring Riser
        * PCO2WC: pCO2 Water (40 meters) (refdes = GI01SUMO-RII11-02-PCO2WC051)
        * CTDMOQ: CTD (40 meters) (refdes = GI01SUMO-RII11-02-CTDMOQ031)
        * PCO2WC: pCO2 Water (80 meters) (refdes = GI01SUMO-RII11-02-PCO2WC052)
        * CTDBPP: CTD (80 meters) (refdes = GI01SUMO-RII11-02-CTDBPP032)
        * PCO2WC: pCO2 Water (130 meters) (refdes = GI01SUMO-RII11-02-PCO2WC053)
        * CTDBPP: CTD (130 meters) (refdes = GI01SUMO-RII11-02-CTDBPP033)
        * PHSENE: Seawater pH (20 meters) (refdes = GI01SUMO-RII11-02-PHSENE041)
        * CTDMOQ: CTD (20 meters) (refdes = GI01SUMO-RII11-02-CTDMOQ011)
        * PHSENE: Seawater pH (100 meters) (refdes = GI01SUMO-RII11-02-PHSENE042)
        * CTDMOQ: CTD (100 meters) (refdes = GI01SUMO-RII11-02-CTDMOQ013)
* GI03FLMA: Flanking Subsurface Mooring A
    * RIS01: Mooring Riser
        * PHSENF: Seawater pH (refdes = GI03FLMA-RIS01-04-PHSENF000)
        * CTDMOG: CTD (30 meters) (refdes = GI03FLMA-RIM01-02-CTDMOG040)
* GI03FLMB: Flanking Subsurface Mooring B
    * RIS01: Mooring Riser
        * PHSENF: Seawater pH (refdes = GI03FLMB-RIS01-04-PHSENF000)
        * CTDMOG: CTD (30 meters) (refdes = GI03FLMB-RIM01-02-CTDMOG060)
        
---
## Surface Mooring